### **LLamaindex and SQLite3**

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_excel(r"C:\Users\AkshayRedekar\OneDrive - VE3\Documents\POC-EXCEL\electronic_data.xlsx")

In [3]:
data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700.00,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [4]:
import sqlite3
conn = sqlite3.connect(":memory:")
data.to_sql("electronics", conn, index=False, if_exists="replace")


200

In [5]:
import pandas as pd
import sqlite3

data = pd.read_excel(r"C:\Users\AkshayRedekar\OneDrive - VE3\Documents\POC-EXCEL\electronic_data.xlsx")
conn = sqlite3.connect(":memory:")
data.to_sql("electronics", conn, index=False, if_exists="replace")


200

In [6]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    tname = table[0]
    cursor.execute(f"PRAGMA table_info({tname});")
    schema = cursor.fetchall()
    print(f"\n🧱 Table: {tname}")
    for col in schema:
        print(f" - {col[1]} ({col[2]})")



🧱 Table: electronics
 - Order ID (INTEGER)
 - Product (TEXT)
 - Quantity Ordered (INTEGER)
 - Price Each (REAL)
 - Order Date (TEXT)
 - Purchase Address (TEXT)


In [7]:
# ! uv pip install llama-index
# ! uv pip install llama-index-llms-groq
# ! uv pip install llama-index-readers-pandas-ai
# ! uv pip install llama-index-storage-chat-store-sqlite
# ! uv pip install pandas openpyxl numpy python-dotenv pysqlite3-binary
# ! uv pip install tqdm

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
import pandas as pd
import sqlite3

excel_path = r"C:\Users\AkshayRedekar\OneDrive - VE3\Documents\POC-EXCEL\electronic_data.xlsx"
data = pd.read_excel(excel_path)

conn = sqlite3.connect(":memory:")
data.to_sql("electronics", conn, index=False, if_exists="replace")


200

In [11]:
import os
import pandas as pd
import sqlite3
from llama_index.llms.groq import Groq
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings, SQLDatabase, VectorStoreIndex, Document

from dotenv import load_dotenv

load_dotenv()

# Initialize models
llm = Groq(model="openai/gpt-oss-20b")
embed_model = GeminiEmbedding(model_name="models/embedding-001")

# Configure LlamaIndex settings
Settings.llm = llm
Settings.embed_model = embed_model

# Load Excel data into SQLite
df = pd.read_excel(r"C:\Users\AkshayRedekar\OneDrive - VE3\Documents\POC-EXCEL\electronic_data.xlsx")
conn = sqlite3.connect(":memory:")
df.to_sql("electronics", conn, index=False, if_exists="replace")


ModuleNotFoundError: No module named 'llama_index.embeddings.gemini'

In [ ]:
# Wrap SQLite DB
sql_db = SQLDatabase(conn)

# Build the index
documents = [Document(text=row.to_string()) for _, row in df.iterrows()]
index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = index.as_query_engine()

# Example: schema discovery
schema_summary = query_engine.query("List all tables and their columns in detail.")
print(schema_summary)

# Example: user question
response = query_engine.query("What is the most expensive product?")
print(response)